In [ ]:
stamppath = '/soft/bio/structure/stamp.4.4/bin/linux/'
slurmpath = '/path/to/slurm/binaries/'
%env STAMPDIR=/soft/bio/structure/stamp.4.4/defs
import os
path= os.environ['PATH']
%env PATH=$stamppath:$path:$slurmpath

import htmd
from htmd import *
from htmd.util import opm
from numpy import *
from glob import glob
from htmd.protocols.equilibration_v2 import Equilibration
from htmd.protocols.production_v4 import Production
from htmd.builder.builder import removeLipidsInProtein, tileMembrane, minimalRotation,removeAtomsInHull
from htmd.rotationmatrix import rotationMatrix
from htmd.builder.charmm import _recoverProtonations
from htmd.molecule.util import sequenceID
import subprocess
import re
from shutil import copy2,copytree,rmtree
htmd.config(viewer='vmd')

In [ ]:
pdbpath = '/path/to/input/PDB_file/structures/folder'
resultspath = '/path/for/output/folder'
stamp_res_dir = resultspath+'/stamp'
opm_pdbid_list_file = './opm_pdb_codes.txt'
topparpath = './toppar'
membranepdb = './membrane/popc36_box_renumbered.pdb'
dummy_atom_name = 'DUM'
dummypdb='./dummy.pdb'
#bash_script_folder = '/path/with/scripts/for/slurm/queuing/system/bash/scripts'
membrane_lipid_segid = 'MEM'
pdblist = glob(pdbpath+'/*/*.pdb')
coldist = 1.3
buffer = 2.4
water_thickness = 20
membrane_distance = 20
water_margin = 4
toposfilenames = ['top_all36_prot.rtf','top_all36_na.rtf','top_all36_lipid.rtf','top_all36_carb.rtf',\
                  'top_all35_ethers.rtf','top_all36_cgenff.rtf']
ligand_toposfilenames = ['/path/to/file.rtf']
ligand_paramsfilenames = ['/path/to/file.prm']
topostreams = ['toppar_water_ions_1.rtf']
paramsfilenames = ['par_all36m_prot.prm','par_all36_na.prm','par_all36_lipid.prm','par_all36_carb.prm',\
                  'par_all35_ethers.prm','par_all36_cgenff.prm']
paramsstreams = ['toppar_water_ions_1.inp','toppar_water_ions_2.inp']
streams_folder='stream_splits'
topos = [os.path.join(topparpath,file) for file in toposfilenames] + \
[os.path.join(os.path.join(topparpath,streams_folder),file) for file in topostreams] + \
ligand_toposfilenames
params = [os.path.join(topparpath,file) for file in paramsfilenames] + \
[os.path.join(os.path.join(topparpath,streams_folder),file) for file in paramsstreams] + \
ligand_paramsfilenames
caps_receptor = ['first ACE', 'last CT3']
caps_not_receptor_protein = ['first NTER', 'last CTER']
capre = re.compile('\s*(\S+)\s+(\S+)\s*')
first = None
last = None
for cap in caps_receptor:
    m = capre.match(cap)
    captype,capres = m.groups()
    if captype == 'first':
        first = capres
    elif captype == 'last':
        last = capres
        

In [ ]:
default_opm_filter = 'not protein or resid > 1000'
pdbid_opm_filters = dict()
for pdbid in {'4lde','4ldl','4ldo','4qkx','5jqh'}:
    pdbid_opm_filters[pdbid] = 'not protein or resid < 1029'
for pdbid in {'5cgc','5cgd'}:
    pdbid_opm_filters[pdbid] = 'not protein or resid < 570'
for pdbid in {'4n4w','4jkv'}:
    pdbid_opm_filters[pdbid] = 'not protein or resid < 190'
for pdbid in {'2ydv','2ydo'}:
    pdbid_opm_filters[pdbid] = 'not protein or resid > 318'

In [ ]:
#Read a file where each line is a set of PDB IDs with similar structure for alignment with OPM structure
#PDB IDs are separated by comma. The sets are separated by new line. The first element of the set is the
#structure to be used if the specified PDB ID is not available in OPM.

def get_opm_eq_dict(opm_pdbid_list_file):
    with open(opm_pdbid_list_file,'r') as f:
        content = f.readlines()
    opm_eq_dict = dict()
    for line in content:
        pdbids = line.split(',')
        ref_pdbid = pdbids[0].strip().lower()
        for pdbid in pdbids:
            opm_eq_dict[pdbid.strip().lower()] = ref_pdbid
    return opm_eq_dict
opm_eq_dict = get_opm_eq_dict(opm_pdbid_list_file)

In [ ]:
pdbfile = pdblist[0]
pdbid = '3pbl'
opm_chain = 'A'
opm_from_resid = 32
opm_to_resid = 400
new_pdb_chain = 'A'

In [ ]:
membranemol = Molecule(membranepdb)


In [ ]:
def define_equilibration():
    md = Equilibration()
    md.runtime = 40
    md.timeunits = 'ns'
    md.temperature = 310
    md.useconstantratio = True
    md.nvtsteps = 0
    md.constraintsteps = 7500000
    md.acemd.minimize = str(5000)
    md.acemd.restart = 'off'
    md.acemd.timestep = '2'
    tcl = """

    proc calcforces_init {} {
        berendsenpressure  off
    }
    proc calcforces {} {
        global numsteps nvtsteps
        set numsteps_4 [expr {$numsteps/4}] 
        ##EQUIL
        set step [ getstep ]
        if { $step > $nvtsteps } {
            berendsenpressure  on
        } else {
            berendsenpressure  off
        }
        if { $step > $numsteps_4 * 3 } {
            constraintscaling 0
        } elseif {$step > $numsteps_4*2} {
            constraintscaling [expr {2.90 - 0.95* $step / $numsteps_4}] 
        }
    }
    proc calcforces_endstep { } { }
    """

    md.acemd.TCL = (md.acemd.TCL[0],tcl)
    return md


In [ ]:
#prepare OPM for STAMP
opmpdbid = pdbid
try:
    opmprot, thickness = get_opm(pdbid,opm_eq_dict)
except Exception:
    opmpdbid = opm_eq_dict[pdbid]
    opmprot, thickness = get_opm(opmpdbid,opm_eq_dict)
#filter opm PDB
opmprot.filter('chain '+opm_chain)
if pdbid in pdbid_opm_filters:
    opm_sel_to_remove = pdbid_opm_filters[pdbid]
else:
    opm_sel_to_remove = default_opm_filter
opmprot.remove(opm_sel_to_remove)
os.makedirs(stamp_res_dir,exist_ok=True)
opmdomainalias = opmpdbid+'_opm_domain'
opmpdbstampfile = stamp_res_dir+'/'+opmpdbid+'_opm.pdb'
opmprot.write(opmpdbstampfile)

In [ ]:
def fix_and_prepare_input(inputmol,first='NTER',last='CTER'):
    mol = inputmol.copy()
    aa= 'ALA ARG ASN ASP CYS GLU GLN GLY HIS ILE LEU LYS MET PHE PRO SER THR TRP TYR VAL'
    mol.set('segid','WAT',sel='water')
    mol.set('resname','TIP3',sel='water')
    mol.set('chain','X',sel='resname TIP3')
    mol.set('name','OH2',sel='resname TIP3 and name OW')
    mol.set('name','H1',sel='resname TIP3 and name HW1')
    mol.set('name','H2',sel='resname TIP3 and name HW2')
    mol.remove('(protein or resname '+aa+') and name O1 O2',_logger=False)
    if first == 'NTER':
        mol.set('name','HT1',sel='(protein or resname '+aa+')and name H1')
        mol.set('name','HT2',sel='(protein or resname '+aa+') and name H2')
        mol.set('name','HT3',sel='(protein or resname '+aa+') and name H3')
    else:
        mol.remove('(protein or resname '+aa+')and name H1 H2 H3',_logger=False)
    if last in {'CTER','CNEU','CTP','CT1'}:
        mol.set('name','OT1',sel='(protein or resname '+aa+') and name O1',_logger=False)
        mol.set('name','OT2',sel='(protein or resname '+aa+') and name O2',_logger=False)
        #fix
        mol.remove('(protein or resname '+aa+') and name OT2',_logger=False)
    else:
        mol.set('name','O',sel='(protein or resname '+aa+') and name O1')
        mol.remove('(protein or resname '+aa+') and name O2',_logger=False)
    mol.set('name','HG1',sel='resname CYS and name HG')
    mol.set('name','HN',sel='resname HIS and name H')
    
    his_he_resids = mol.get('resid',sel='resname HIS and name HE2')
    his_he_chains = mol.get('chain',sel='resname HIS and name HE2')
    his_he_ids = set([':'.join((chain,str(resid))) for resid,chain in zip(his_he_resids,his_he_chains)])
    his_hd_resids = mol.get('resid',sel='resname HIS and name HD1')
    his_hd_chains = mol.get('chain',sel='resname HIS and name HD1')
    his_hd_ids = set([':'.join((chain,str(resid))) for resid,chain in zip(his_hd_resids,his_hd_chains)])
    hsd_ids = his_hd_ids.difference(his_he_ids)
    hse_ids = his_he_ids.difference(his_hd_ids)
    hsp_ids = his_hd_ids.intersection(his_he_ids)
    
    hsd_dict = dict()
    hse_dict = dict()
    hsp_dict = dict()
    
    for chain,resid in [ id1.split(':') for id1 in hsd_ids]:
        if chain not in hsd_dict:
            hsd_dict[chain] = []
        hsd_dict[chain].append(resid)
    for chain,resid in [ id1.split(':') for id1 in hse_ids]:
        if chain not in hse_dict:
            hse_dict[chain] = []
        hse_dict[chain].append(resid)
    for chain,resid in [ id1.split(':') for id1 in hsp_ids]:
        if chain not in hsp_dict:
            hsp_dict[chain] = []
        hsp_dict[chain].append(resid)
        
    for chain in hsd_dict:
        sel1 = 'resname HIS and chain '+chain+' and resid '+' '.join(hsd_dict[chain])
        mol.set('resname','HSD',sel=sel1)
    for chain in hse_dict:
        sel1 = 'resname HIS and chain '+chain+' and resid '+' '.join(hse_dict[chain])
        mol.set('resname','HSE',sel=sel1)
    for chain in hsp_dict:
        sel1 = 'resname HIS and chain '+chain+' and resid '+' '.join(hsp_dict[chain])
        mol.set('resname','HSP',sel=sel1)
    mol = autoSegment(mol,sel='protein or resname '+aa)
    mol.set('segid','LIG',sel='not (protein or resname '+aa+') and not water and not ions')
    mol.set('chain','L',sel='segid LIG')
    mol.set('segid','ION',sel='ions')
    mol.set('chain','N',sel='segid ION')
    protsegids = set(mol.get('segid',sel='protein'))
    mol = renumber_resid(mol,'water',by=1)
    mol = renumber_resid(mol,'ions',by=1)
    mol = renumber_resid(mol,'segid LIG',by=1)
    return (mol,protsegids)
    #    for segid in protsegids:
    #        resids = set(mol.get('resid',sel='segid '+segid))
    #        for resid in resids:
    #            resname = mol.get('resname',sel='resid '+str(resid)+' and segid '+segid)[0]
    #            chain = mol.get('chain',sel='resid '+str(resid)+' and segid '+segid)[0]
    #            mol.set('segid',segid,sel='resname '+resname+' and chain '+chain+' and resid '+str(segid))

In [ ]:
def write_stamp_domain(pdbstampfile,stamp_res_dir,pdbdomainalias,pdb_chain,pdb_from_resid,pdb_to_resid):
    os.makedirs(stamp_res_dir,exist_ok=True)
    pdbdomain = "%s %s {%s %d _ to %s %d _}\n" % (pdbstampfile,pdbdomainalias,pdb_chain,pdb_from_resid,pdb_chain,pdb_to_resid)
    pdbdomainrootname = stamp_res_dir+'/'+pdbdomainalias
    pdbdomainfile = pdbdomainrootname+'.domain'
    with open(pdbdomainfile,'w') as f:
        f.write(pdbdomain)
    
    return pdbdomainfile

def stamp_align(stamp_res_dir,pdbstampfile,pdbdomainalias,pdb_chain,pdb_from_resid,pdb_to_resid,\
                refpdbstampfile,refpdbdomainalias,refpdb_chain,refpdb_from_resid,refpdb_to_resid,min_quality=6):
    os.makedirs(stamp_res_dir,exist_ok=True)
    pdbdomainrootname = stamp_res_dir+'/'+pdbdomainalias
    pdbdomainprefix = pdbdomainrootname+'_prot'
    refpdbdomainfile = write_stamp_domain(refpdbstampfile,stamp_res_dir,refpdbdomainalias,refpdb_chain,refpdb_from_resid,refpdb_to_resid)

    pdbdomainfile = write_stamp_domain(pdbstampfile,stamp_res_dir,pdbdomainalias,pdb_chain,pdb_from_resid,pdb_to_resid)

    #scan sequences
    process1=subprocess.run(('stamp','-l',refpdbdomainfile,'-n',str(2),'-s','-slide',str(5),'-d',pdbdomainfile,'-prefix',pdbdomainprefix),stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=False)
    with open(pdbdomainprefix+'_scan.log','w') as f:
        f.write(process1.stdout.decode())
    with open(pdbdomainprefix+'_scan_err.log','w') as f:
        f.write(process1.stderr.decode())
    process1.check_returncode()

    #remove poor quality aligments
    process2=subprocess.run(('sorttrans','-f',pdbdomainprefix+'.scan','-s','Sc',str(min_quality)),stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=False)
    with open(pdbdomainprefix+'.sorted','bw') as f:
        f.write(process2.stdout)
    with open(pdbdomainprefix+'_sort_err.log','w') as f:
        f.write(process2.stderr.decode())
    process2.check_returncode()

    #STAMP alignment    
    process3=subprocess.run(('stamp','-l',pdbdomainprefix+'.sorted','-prefix',pdbdomainprefix),stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=False)
    with open(pdbdomainprefix+'_stamp.log','bw') as f:
        f.write(process3.stdout)
    with open(pdbdomainprefix+'_stamp_err.log','w') as f:
        f.write(process3.stderr.decode())
    process3.check_returncode()

    current_wd=os.getcwd()
    #Coordinate tranformation
    os.chdir(stamp_res_dir)
    outputpdb = pdbdomainrootname+'_1.pdb'
    try:
        os.unlink(outputpdb)
    except Exception:
        pass
    process4=subprocess.run(('transform','-f',pdbdomainprefix+'.1'),stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=False)
    os.chdir(current_wd)
    with open(pdbdomainprefix+'_transf.log','bw') as f:
        f.write(process4.stdout)
    with open(pdbdomainprefix+'_transf_err.log','w') as f:
        f.write(process4.stderr.decode())
    try:
        os.unlink(stamp_res_dir+'/'+refpdbdomainalias+'.pdb')
    except Exception:
        pass
    process4.check_returncode()
    return outputpdb


In [ ]:
def renumber_resid(inputmol,sel,by=3):
    mol = inputmol.copy()
    option_num = 2
    max_value = 2**option_num - 1
    if by > max_value:
        raise ValueError('Maximum value for "by" keyword is %d.' % max_value)
    if by < 1:
        raise ValueError('Minimum value for "by" keyword is "1".')
    bin_by = format(by,'0'+str(option_num)+'b')
    option_array = [bool(int(i)) for i in bin_by]
    by_segid = option_array[0]
    by_resname = option_array[1]
    if by_segid:
        segids = set(mol.get('segid',sel=sel))      
        for segid in segids:
            if by_resname:
                resnames = set(mol.get('resname',sel='(%s) and segid %s' % (sel,segid)))
                for resname in resnames:
                    lsel = '(%s) and (segid %s) and (resname %s)' % (sel,segid,resname)
                    mol = renumber_resid_by_resid(lsel,mol)
            else:
                lsel = '(%s) and (segid %s)' % (sel,segid)
                mol = renumber_resid_by_resid(lsel,mol)
    else:                      
        resnames = set(mol.get('resname',sel=sel))
        for resname in resnames:
            lsel = '(%s) and (resname %s)' % (sel,resname)
            mol = renumber_resid_by_resid(lsel,mol)
    return mol

def renumber_resid_by_resid(sel,inputmol,ordered=False):
    mol = inputmol.copy()
    resids = list(set(mol.get('resid',sel=sel)))
    if ordered:
        resids = sort(resids)
    newresid = 1
    for resid in resids:
        mol.set('resid',newresid,sel='(%s) and (resid %s)' % (sel,resid))
        newresid += 1
    return mol

def prepare_pdbmol(inputpdbmol):
    pdbmol = inputpdbmol.copy()
    pdbmol = renumber_resid(pdbmol,'water',by=1)
    pdbmol = renumber_resid(pdbmol,'ions',by=1)
    pdbmol = renumber_resid(pdbmol,'not protein and not water and not ions',by=1)
    pdbmol = autoSegment(pdbmol,receptorsel)
    receptor_segids = set(pdbmol.get('segid',sel=receptorsel))
    not_receptor_protein_sel = 'protein and not ('+receptorsel+')'
    if len(pdbmol.get('index',sel=not_receptor_protein_sel)) > 0:
        pdbmol = autoSegment(pdbmol,not_receptor_protein_sel,basename='Q')
        not_receptor_protein_segids = set(pdbmol.get('segid',sel=not_receptor_protein_sel))
    else:
        not_receptor_protein_segids = set()
    
    pdbmol.set('segid','WAT',sel='water')
    pdbmol.set('segid','ION',sel='ions')
    pdbmol.set('segid','LIG',sel='not protein and not water and not ions')
    return pdbmol, receptor_segids, not_receptor_protein_segids

In [ ]:
def add_membrane(pdbmol,membranemol,protsegids,membrane_distance,coldist=1.3):
    # Corrections for rotational difusion
    prot = pdbmol.copy()
    protsel = 'segid '+' '.join(protsegids)
    prot.filter(protsel,_logger=False)
    r = minimalRotation(prot)
    M = rotationMatrix([0, 0, 1], r)
    pdbmol.rotateBy(M)
    pcoor = pdbmol.get('coords',sel=protsel)
    Mcoor = np.max(pcoor,axis=0)
    mcoor = np.min(pcoor,axis=0)
    # get the diagonal of XY of the protein if XY is a square 
    # which side is as long as the largest side (between X and Y) from the protein box  
    p_dim = [Mcoor[0] - mcoor[0],Mcoor[1] - mcoor[1]]
    maxXY = sqrt(p_dim[0]**2+p_dim[1]**2)
    minimum_box_size_x = maxXY+2
    minimum_box_size_y = minimum_box_size_x
    
    
    # get min max coor of the system
    minc = np.min(pdbmol.coords, axis=0).flatten()
    maxc = np.max(pdbmol.coords, axis=0).flatten()
    
    system_size_x = maxc[0] - minc[0]
    system_size_y = maxc[1] - minc[1]
    
    center_x = system_size_x/2 + minc[0]
    center_y = system_size_y/2 + minc[1]
    
    system_size = np.max([system_size_x,system_size_y])
    print(str(minimum_box_size_x),str(system_size))
    corr_system_size_x = np.max([minimum_box_size_x,system_size]) 
    corr_system_size_y = np.max([minimum_box_size_y,system_size])
    
    addmembdist = membrane_distance/2.0+np.max([coldist,1.5])+0.0
    
    xlim = [center_x-corr_system_size_x/2-addmembdist,center_x+corr_system_size_x/2+addmembdist]
    ylim = [center_y-corr_system_size_y/2-addmembdist,center_y+corr_system_size_y/2+addmembdist]
    
    memb = membranemol.copy()
    memb.remove('ions',_logger=False)
    memb2 = tileMembrane(memb, xlim[0], ylim[0], xlim[1], ylim[1])
    
    #from tileMembrane
    size = np.max(membranemol.get('coords', 'water'), axis=0) - np.min(membranemol.get('coords', 'water'), axis=0)
    xreps = int(np.ceil((xlim[1] - xlim[0]) / size[0]))
    yreps = int(np.ceil((ylim[1] - ylim[0]) / size[1]))
    
    membtmp_segids = ordered_unique(memb2.get('segid'))
    k=0
    for segid in membtmp_segids:
    #    memb2.set('segid','M'+str(k),sel='segid '+segid+' and not waters')
         memb2.set('segid','W'+str(k),sel='segid '+segid+' and waters')
         k += 1
    
    memb2 = renumber_segments(memb2,set(memb2.get('segid',sel='waters')),'MW')
    memb2 = renumber_segments(memb2,set(memb2.get('segid',sel='not waters')),'MEM')
    membrane_resnames = set(memb2.get('resname'))
    membrane_segids = set(memb2.get('segid'))
    
    mcenter = np.mean(memb2.get('coords',sel='segid MEM'),axis=0)
    memb2.moveBy(-mcenter)

    memb2, num = removeLipidsInProtein(pdbmol, memb2,lipidsel='lipids or waters')
    
    mol = pdbmol.copy()
    mol.append(memb2, collisions=True,coldist=coldist)
    
    return (mol, membrane_resnames,membrane_segids,xreps,yreps)


In [ ]:
def solvate_pdbmol(mol,membrane_segids,water_thickness,water_margin,buffer=2.4,coldist=1.3,prefix='W'):
    waterbox = mol.get('coords','(waters or ions) and segid '+' '.join(membrane_segids))
    mwaterbox = np.min(waterbox, axis=0)
    Mwaterbox = np.max(waterbox, axis=0)
    coo = mol.get('coords','not (waters or ions)')
    mcoo = np.min(coo, axis=0)
    Mcoo = np.max(coo, axis=0)
    cooall = mol.get('coords','all')
    mcooall = np.min(coo, axis=0)
    Mcooall = np.max(coo, axis=0)
    #top layer
    M1z = Mcoo[2] + water_thickness/2. + np.max((coldist,buffer)) - buffer
    m1z = Mwaterbox[2] - water_margin
    M1 = [Mwaterbox[0],Mwaterbox[1],M1z]
    m1 = [mwaterbox[0],mwaterbox[1],m1z]
    
    #bottom layer
    M2z = mwaterbox[2] + water_margin
    m2z = mcoo[2] - water_thickness/2.- np.max((coldist,buffer)) + buffer
    M2 = [Mwaterbox[0],Mwaterbox[1],M2z]
    m2 = [mwaterbox[0],mwaterbox[1],m2z]

    smol = solvate(mol, minmax=np.vstack((m2,M1)),prefix=prefix,buffer=buffer)

    smol, num_remove = removeAtomsInHull(smol, smol, 'name CA', 'segid "'+prefix+'[0-9]+"')
    #wtsegids = set(smol.get('segid',sel='segid "%s.*"'% prefix))
    #for segid in wtsegids:
        #smol.remove('segid %s and same resid as ( z > %g and z < %g)' % (segid,M2[2],m1[2]),_logger=False)

    return smol

In [ ]:

dummymol = Molecule(dummypdb)
dummy_sel = 'name '+dummy_atom_name
def add_dummy_atom(inputmol,property_dict):
    mol = inputmol.copy()
    dummymol1 = dummymol.copy()
    for prop in property_dict:
        dummymol1.set(prop,property_dict[prop])
    mol.append(dummymol1,coldist=None)
    return mol
def add_center_dummy_atom(inputmol,coords,property_dict):
    mol = inputmol.copy()
    center = np.mean(coords,axis=0)
    property_dict['coords'] = center
    mol = add_dummy_atom(mol,property_dict)
    return mol
def remove_aromatic_insertions(inputmol,protsegids,coldist=1.5,outpdb=None):
    mol = inputmol.copy()
    atoms_data = [['TRP','CG CD1 CE1 NE1 CE2 CD2',5,'1'],
                 ['TRP','CD2 CE2 CZ2 CH2 CZ3 CE3',6,'2'],
                 ['PRO','N CA CB CG CD',5,''],
                 ['HIS HSD HSE HSP HID HIE HIP',
                  'CG CD1 CE1 CZ CE2 CD2 ND1 NE2',5,''],
                 ['PHE TYR TYM',
                  'CG CD1 CE1 CZ CE2 CD2',6,'']]
    beta_backup = np.copy(mol.beta)
    mol.set('beta',sequenceID((mol.resid, mol.insertion, mol.segid)))    
    
    for atom_data in atoms_data:
        atom_step = atom_data[2]
        suffix = atom_data[3]
        sel = 'resname %s and name %s' % (atom_data[0],atom_data[1])
        idxs = mol.get('index',sel=sel)
        resnames = mol.resname[idxs]
        resids = mol.resid[idxs]
        segids = mol.segid[idxs]
        coords = mol.coords[idxs,:,0]
        atom_num = len(idxs)
        if atom_num % atom_step != 0:
            raise ValueError('Missing atoms.')
        for i in range(0,atom_num,atom_step):
            property_dict = {'resname':resnames[i]+suffix,'resid':resids[i],'segid':segids[i]}
            mol = add_center_dummy_atom(mol,coords[i:i+atom_step],property_dict)
            
    if outpdb:
        mol.write(outpdb)
    var_list = tuple([coldist]+[dummy_sel for i in range(0,3)])
    
    dummy_atoms_idxs = mol.get('index',sel=dummy_sel)
    dummy_atoms_resid = mol.resid[dummy_atoms_idxs]
    dummy_atoms_segid = mol.segid[dummy_atoms_idxs]
    removed_indexes = []
    for idx,resid,segid in zip(dummy_atoms_idxs,dummy_atoms_resid,dummy_atoms_segid):
        r_idx1 = smol.remove('same beta as ((exwithin %g of (index %d)) and not (resid %d and segid %s))' \
                             % (coldist,idx,resid,segid),_logger=False)
        removed_indexes = removed_indexes + r_idx1.tolist()
    mol.remove(dummy_sel,_logger=False)
    inv_idx1 = np.setdiff1d(np.arange(len(beta_backup)), np.array(removed_indexes), assume_unique=True)
    mol.beta = beta_backup[inv_idx1]

    
    if len(removed_indexes) > 0:
        print('WARNING: removed '+str(len(removed_indexes))+' atoms within '+str(coldist)+' of a protein aromatic ring')
    
    return (mol,removed_indexes)

In [ ]:
def renumber_resid_vmd(mol,sel,by=3,start=1):
    import tempfile
    tmpin = tempfile.NamedTemporaryFile(suffix='.pdb')
    mol.write(tmpin.name)
    viewer = getCurrentViewer(dispdev='text')
    viewer.send('set molid [mol new {%s}]' % tmpin.name)
    tmpin.close()
    tmpout = tempfile.NamedTemporaryFile(suffix='.pdb')
    option_num = 2
    max_value = 2**option_num - 1
    if by > max_value:
        raise ValueError('Maximum value for "by" keyword is %d.' % max_value)
    if by < 1:
        raise ValueError('Minimum value for "by" keyword is "1".')
    bin_by = format(by,'0'+str(option_num)+'b')
    option_array = [bool(int(i)) for i in bin_by]
    by_segid = option_array[0]
    by_resname = option_array[1]
   
    
    if by_segid:
        segids = set(mol.get('segid',sel=sel))      
        for segid in segids:
            if by_resname:
                resnames = set(mol.get('resname',sel='(%s) and segid %s' % (sel,segid)))
                for resname in resnames:
                    lsel = '(%s) and (segid %s) and (resname %s)' % (sel,segid,resname)
                    viewer = renumber_resid_by_resid_vmd(lsel,mol,viewer,start=start)
            else:
                lsel = '(%s) and (segid %s)' % (sel,segid)
                viewer = renumber_resid_by_resid_vmd(lsel,mol,viewer,start=start)
    else:                      
        resnames = set(mol.get('resname',sel=sel))
        for resname in resnames:
            lsel = '(%s) and (resname %s)' % (sel,resname)
            viewer = renumber_resid_by_resid_vmd(lsel,mol,viewer,start=start)       
    viewer.send('animate write pdb {%s} waitfor all top;exit' % tmpout.name)
    newmol = Molecule(tmpout.name)
    tmpout.close()
    return newmol

def renumber_resid_by_resid_vmd(sel,mol,viewer,start=1):
    resids = sorted(list(set(mol.get('resid',sel=sel))))
    resids = [str(i) for i in resids]
    viewer.send('proc renum_resid {molid} {set newresid %d; set resids {%s};' % (start,' '.join(resids)) + \
                'set asall [atomselect $molid [concat {(%s) and resid } $resids]];' % sel + \
                '$asall set user 1.00;' + \
                'foreach resid $resids {' + \
                'set as [atomselect $molid [concat {user 1.00 and (%s) and resid } $resid]];' % sel + \
                '$as set resid $newresid; $as set user 0.00; incr newresid}};'+'renum_resid $molid')
    return viewer
def ordered_unique(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]
def renumber_segments(inputmol,segids,prefix):
    sel = 'segid '+' '.join(segids)
    segids = ordered_unique(inputmol.get('segid',sel=sel))
    if prefix in segids:
        raise ValueError('segid %s already exists.' % prefix)
        
    mol = renumber_resid_vmd(inputmol,sel,by=2)
    # change first segid segment as it is properly renumbered already
    mol.set('segid',prefix,sel='segid '+segids[0])

    if len(segids) > 1:
        # initialize variables for second segid
        curr_segid = prefix
        # get last resid for first segid
        idx_curr_segid = mol.atomselect('segid '+curr_segid)
        prev_resid = len(set(mol.resid[idx_curr_segid]))
        k = 0
        for segid in segids[1:]:
            
            # get last current resid
            idx_segid = mol.atomselect('segid '+segid)
            curr_resid = len(set(mol.resid[idx_segid])) + prev_resid
            if curr_resid <= 9999:
                # join segments resuming the previous resid numbering
                mol = renumber_resid_vmd(mol,'segid '+segid,start=prev_resid+1,by=2)
                mol.segid[idx_segid] = curr_segid
                # get last resid of the current segid for the next loop iteration
                prev_resid = curr_resid
            else:

                # join segments resuming the previous resid numbering up to resid 9999
                sel1 = 'segid '+segid+' and resid <= '+str(9999-prev_resid)
                mol = renumber_resid_vmd(mol,sel1,start=prev_resid+1,by=2)
                mol.set('segid',curr_segid,sel=sel1)
                # define next new segment with resids > 9999
                k +=1
                curr_segid = prefix+str(k)
                if curr_segid in segids:
                    raise ValueError('segid %s already exists.' % curr_segid)
                # resid <= 9999 still preserve the old segid
                idx_curr_segid = mol.atomselect('segid '+segid)
                mol.segid[idx_curr_segid] = curr_segid
                # get last resid of the current segid for the next loop iteration
                mol = renumber_resid_vmd(mol,'segid '+curr_segid,by=2)
                prev_resid = len(set(mol.resid[idx_curr_segid]))
            
        if k > 0:
            if prefix+str(0) in segids:
                print('WARNING: segid %s already exists, using %s instead.' % (prefix,prefix+str(0)))
            else:
                mol.segid[mol.segid == prefix] = prefix+str(0)
        
    return mol

In [ ]:
# run me for building
# for equilibration protocol look at define_equilibration
for pdbfile in pdblist:
    pdbmol = Molecule(pdbfile)
    modelname = os.path.basename(os.path.dirname(pdbfile))
    min_quality = 6

    print('Start of '+modelname)
    pdbmol,receptor_segids = fix_and_prepare_input(pdbmol,first=first,last=last)
    not_receptor_protein_segids = set()
    #prepare PDB for STAMP
    receptorsel = 'segid '+' '.join(receptor_segids)
    protresids = set(pdbmol.get('resid',sel=receptorsel))
    pdb_from_resid = np.min(list(protresids))
    pdb_to_resid = np.max(list(protresids))
    pdbmol.set('chain',new_pdb_chain,sel=receptorsel)
    
    pdbstampfile = stamp_res_dir+'/'+modelname+'.pdb'
    pdbdomainalias = modelname
    pdbmol.write(pdbstampfile)

    # Do STAMP
    
    if min_quality is not None:
        print('STAMP...')
        stamp_aligned_pdbfile = stamp_align(stamp_res_dir,pdbstampfile,pdbdomainalias,new_pdb_chain,\
            pdb_from_resid,pdb_to_resid,opmpdbstampfile,opmdomainalias,opm_chain,opm_from_resid,opm_to_resid,min_quality=min_quality)
        stampmol = Molecule(stamp_aligned_pdbfile)
        pdbmol.align(receptorsel,refmol=stampmol)
    
    #Center to receptor XY
    center = np.mean(pdbmol.get('coords',sel=receptorsel),axis=0)
    pdbmol.moveBy([-center[0],-center[1],0])
    
    print('Adding membrane...')
    #Add membrane
    mol, membrane_resnames, membrane_segids, xreps, yreps = add_membrane(pdbmol,membranemol,receptor_segids,membrane_distance)
    
    #Solvate
    print('Solvating...')
    smol = solvate_pdbmol(prebuildmol,membrane_segids,water_thickness,water_margin,buffer=buffer,coldist=coldist,prefix='WT')
    
    print('Pre-build...')
    #Pre-build
    caps = dict()
    for segid in receptor_segids:
        caps[segid] = caps_receptor
    for segid in not_receptor_protein_segids:
        caps[segid] = caps_not_receptor_protein
    prebuildmol = charmm.build(mol, topo=topos, param=params,caps=caps,outdir=resultspath+'/pre-build/'+\
                               modelname,ionize=False)
    prebuild_psffile = prebuildmol.topoloc
    prebuild_pdbfile = os.path.splitext(prebuildmol.topoloc)[0]+'.pdb'
    prebuildmol = Molecule(prebuild_pdbfile)
    _recoverProtonations(prebuildmol)
    

    print('Checking aromatic insertions...')
    smol,removed_indexes = remove_aromatic_insertions(smol,receptor_segids,outpdb=resultspath+'/pre-build/'+modelname+'/aromatic_check.pdb')
    
    lipid_num = len(set(smol.get('resid',sel='segid '+membrane_lipid_segid)))
    solv_num = len(smol.get('index',sel='resname TIP3 and name OH2'))
    if float(solv_num) / lipid_num < 35:
        raise ValueError('Water/lipid ratio lower than 35.')
    
    
    print('Renumbering...')
    smol = renumber_resid_vmd(smol,'segid '+' '.join(membrane_segids),by=2)
    print('Ionizing...')
    molbuilt = charmm.build(smol, topo=topos, param=params,
                            outdir=resultspath+'/ionize/'+modelname,
                            saltconc=0.15,caps=caps)
    build_psffile = molbuilt.topoloc
    build_pdbfile = os.path.splitext(molbuilt.topoloc)[0]+'.pdb'
    molbuilt = Molecule(build_pdbfile)
    _recoverProtonations(molbuilt)
    print('Building...')
    molbuilt = renumber_resid_vmd(molbuilt,'segid "WT.*" or segid I',by=2)
    molbuilt = charmm.build(molbuilt, topo=topos, param=params,
                            outdir=resultspath+'/build/'+modelname,
                            caps=caps,ionize=False)
    md = define_equilibration()
    md.constraints = {'segid '+' '.join(receptor_segids)+' and name C CA N O or not (segid ' + \
                  ' '.join(receptor_segids)+' or resname '+' '.join(membrane_resnames) + \
                  ' or water or ions ) and noh or segid ION WAT and noh':1.0}
    md.write(resultspath+'/build/'+modelname,resultspath+'/build/'+modelname+'/equil')
    # This replaces htmd generated run.sh. Input and output for simulations is set in bash_script_folder+'/run_equil.sh'.
    #copy2(bash_script_folder+'/run_equil.sh',resultspath+'/build/'+modelname+'/equil/run.sh')
    #copy2(bash_script_folder+'/copyback.sh',resultspath+'/build/'+modelname+'/equil')
    print('End of '+modelname+'\n')
    

In [ ]:
#run me for lunching equilibration
#equilibration protocol has been defined before build
try:
    sqs
except NameError:
    sqs = {}

for pdbfile in pdblist:
    modelname = os.path.basename(os.path.dirname(pdbfile))
    if modelname in sqs:
        print('Skipping '+modelname+': it has already been submitted.')
        continue

    sq = SlurmQueue()
    sq.jobname = 'jobname'
    sq.datadir = None
    sq.partition = 'partition_name'
    sq.ngpu = 1
    sq.ncpu = 2
    
    #sq.exclude = 'excluded_node'
    
    # directory to copy input and store output of equilibration (initial working directory for run_equil.sh).
    equildir = '/input/folder/for/running/equilibration'+modelname+'/equil/'
    # delete previous equildir
    if os.path.isdir(equildir):
        rmtree(equildir)
    # copy equil folder in build to equildir
    copytree(resultspath+'/build/'+modelname+'/equil',equildir)
    sq.submit(equildir)
    sqs[modelname] = sq

In [ ]:
#run me to check how many simulations are still running
sum([sqs[modelname].inprogress() for modelname in sqs])
    

In [ ]:
# WARNING!!!: run me to KILL simulations that are still running
for modelname in sqs:
    sqs[modelname].stop()

In [ ]:
# reset tracking for all
sqs = {}

In [ ]:
# remove modelnames from tracking (for rerun)
modelnames = {'3pbl','D3'}
for modelname in modelnames:
    sqs.pop(modelname)

In [ ]:
#run me to define production protocol
def define_production():
    md = Production()
    md.runtime = 500
    md.timeunits = 'ns'
    md.temperature = 310
    md.acemd.restart = 'off'
    md.acemd.bincoordinates = 'output.coor'
    md.acemd.tclforces = 'off'
    md.acemd.extendedsystem  = 'output.xsc'
    md.acemd.binvelocities = 'output.vel'
    md.acemd.restartfreq = str(25000)
    md.acemd.berendsenpressure = 'off'
    md.acemd.TCL = (md.acemd.TCL[0],"")
    return md
md = define_production()



In [ ]:
# run me to launch production replicates

# number of replicates
repnum = 3
# manual skip
# e.g. modelname_skip = {'3pbl'}
# no skip: modelname_skip = {}
modelname_skip = {'3pbl'}
pdblist = glob(pdbpath+'/*/*.pdb')
try:
    sqs_p
except NameError:
    sqs_p = {}
for pdbfile in pdblist:
    modelname = os.path.basename(os.path.dirname(pdbfile))
    
    # must match with equildir in equilibration launcher code and contain input and output of equilibration.
    equildir = '/input/folder/for/running/equilibration'+modelname+'/equil/'
    for rep in range(1,repnum+1):
        if modelname in modelname_skip:
            print('Skipping '+modelname+'_'+str(rep)+'.')
            continue
        if modelname+'_'+str(rep) in sqs_p:
            print('Skipping '+modelname+'_'+str(rep)+': it has already been submitted.')
            continue
        md = define_production()
        # directory copy output of equilibration to production input (initial working directory for run_prod.sh).
        inputdir='/input/folder/for/running/production'+modelname+'/input/rep_%d' % rep
        md.write(equildir,inputdir)
        
        # This replaces htmd generated run.sh. Input and output for simulations is set in bash_script_folder+'/run_equil.sh'.
        # Memo 7-9-1017: datadir = inputdir+'/../../data/'+modelname #defined in bash_script_folder+'/run_equil.sh
        #copy2(bash_script_folder+'/run_prod.sh',inputdir+'/run.sh')
        #copy2(bash_script_folder+'/copyback.sh',inputdir)
        
        sq = SlurmQueue()
        sq.jobname = 'Carlsson3_'+modelname+'_prod'
        sq.datadir = None
        sq.partition = 'phi'
        sq.ngpu = 1
        sq.ncpu = 2
        
        # uncomment if you want to exclude bifur
        #sq.exclude = 'bifur'
        
        sqs_p[modelname+'_'+str(rep)] = sq
        sq.submit(inputdir)

In [ ]:
#run me to check how many simulations are still running
sum([sqs_p[modelname_rep].inprogress() for modelname_rep in sqs_p])

In [ ]:
# WARNING!!!: run me to KILL simulations that are still running
for modelname_rep in sqs_p:
    sqs_p[modelname_rep].stop()

In [ ]:
# reset tracking for all
sqs_p = {}

In [ ]:
# remove modelname_replicate from tracking (for rerun)
modelname_replicates = {'3pbl_1','3pbl_2'}
for modelname_rep in modelname_replicates:
    sqs_p.pop(modelname_rep)

In [ ]:
# remove all replicates of a system or systems from tracking (for rerun)
modelnames = ['3pbl','D3']
for modelname in modelnames:
    keys = [key for key in sqs_p if key.startswith(modelname+'_')]
    for key in keys:
        sqs_p.pop(key)

In [ ]:
#rerun an specific system and replicate
#repdict = {'system':rep_id}
rep_dict = {'D3_5334_0002':3}
pdblist = glob(pdbpath+'/D3_5334_0002/*.pdb')
try:
    sqs_p
except NameError:
    sqs_p = {}
for pdbfile in pdblist:
    modelname = os.path.basename(os.path.dirname(pdbfile))
    equildir = '/input/folder/for/running/equilibration'+modelname+'/equil/'
    rep = rep_dict[modelname]
    md = define_production()
    # directory to copy input and store output of equilibration (initial working directory for run_prod.sh).
    inputdir='/input/folder/for/running/production'+modelname+'/input/rep_%d' % rep
    md.write(equildir,inputdir)
    # This replaces htmd generated run.sh. Input and output for simulations is set in bash_script_folder+'/run_equil.sh'.
    # Memo 7-9-1017: datadir = inputdir+'/../../data/'+modelname #defined in bash_script_folder+'/run_equil.sh
    #copy2(bash_script_folder+'/run_prod.sh',inputdir+'/run.sh')
    #copy2(bash_script_folder+'/copyback.sh',inputdir)
    
    sq = SlurmQueue()
    sq.jobname = 'Carlsson3_'+modelname+'_prod'
    sq.datadir = None
    sq.partition = 'phi'
    sq.ngpu = 1
    sq.ncpu = 2
    
    # uncomment if you want to exclude bifur
    #sq.exclude = 'bifur'
    
    sqs_p[modelname+'_'+str(rep)] = sq
    sq.submit(inputdir)